In [1]:
import CoolProp.CoolProp as CP
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from IPython.display import IFrame
fluids = ['Water', 'Ammonia', 'R134a']

for fluid in fluids:
    T_min = CP.PropsSI('Tmin', fluid) + 10  # Adding a small offset to Tmin
    T_max = CP.PropsSI('Tcrit', fluid)
    T_vals = np.linspace(T_min, T_max, 400)  # Number of points

    p_v_liquid = {'Pressure (Pa)': [], 'Specific Volume Liquid (m^3/kg)': []}
    p_v_vapor = {'Pressure (Pa)': [], 'Specific Volume Vapor (m^3/kg)': []}

    for T in T_vals:
        p = CP.PropsSI('P', 'T', T, 'Q', 0, fluid)
        v_liquid = 1 / CP.PropsSI('D', 'T', T, 'Q', 0, fluid)  # Saturated liquid
        v_vapor = 1 / CP.PropsSI('D', 'T', T, 'Q', 1, fluid)  # Saturated vapor
        p_v_liquid['Pressure (Pa)'].append(p)
        p_v_liquid['Specific Volume Liquid (m^3/kg)'].append(v_liquid)
        p_v_vapor['Pressure (Pa)'].append(p)
        p_v_vapor['Specific Volume Vapor (m^3/kg)'].append(v_vapor)

    # Save to CSV
    pd.DataFrame(p_v_liquid).to_csv(f'{fluid}_p_v_liquid.csv', index=False)
    pd.DataFrame(p_v_vapor).to_csv(f'{fluid}_p_v_vapor.csv', index=False)


import plotly.graph_objs as go

for fluid in fluids:
    df_liquid = pd.read_csv(f'{fluid}_p_v_liquid.csv')
    df_vapor = pd.read_csv(f'{fluid}_p_v_vapor.csv')

    # Creating Plotly plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_liquid['Specific Volume Liquid (m^3/kg)'], y=df_liquid['Pressure (Pa)'],
                             mode='lines', name='Saturated Liquid'))
    fig.add_trace(go.Scatter(x=df_vapor['Specific Volume Vapor (m^3/kg)'], y=df_vapor['Pressure (Pa)'],
                             mode='lines', name='Saturated Vapor'))

    # Set log scale for x-axis
    fig.update_layout(xaxis_type="log", title=f'{fluid} p-v Diagram', 
                      xaxis_title='Specific Volume (m^3/kg)', yaxis_title='Pressure (Pa)')

    # Save as HTML
    fig.write_html(f'{fluid}_p_v_plot.html')

